In [2]:
import pandas as pd
# import geopy as geo
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler # Handliing Feature Scaling
from sklearn.impute import SimpleImputer #Handling missing values
from sklearn.preprocessing import OrdinalEncoder #Ordinal Encoding
##Pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [23]:
df = pd.read_csv("https://raw.githubusercontent.com/AKISHPOTHURI/DataScience/main/Dataset/online_order.csv")
# data.head()

In [16]:
df.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
0,0xcdcd,DEHRES17DEL01,36.0,4.2,30.327968,78.046106,30.397968,78.116106,12-02-2022,21:55,22:10,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46
1,0xd987,KOCRES16DEL01,21.0,4.7,10.003064,76.307589,10.043064,76.347589,13-02-2022,14:55,15:05,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23
2,0x2784,PUNERES13DEL03,23.0,4.7,18.562450,73.916619,18.652450,74.006619,04-03-2022,17:30,17:40,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21
3,0xc8b6,LUDHRES15DEL02,34.0,4.3,30.899584,75.809346,30.919584,75.829346,13-02-2022,09:20,09:30,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20
4,0xdb64,KNPRES14DEL02,24.0,4.7,26.463504,80.372929,26.593504,80.502929,14-02-2022,19:50,20:05,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41


In [18]:
df.isnull().sum()

ID                                0
Delivery_person_ID                0
Delivery_person_Age            1854
Delivery_person_Ratings        1908
Restaurant_latitude               0
Restaurant_longitude              0
Delivery_location_latitude        0
Delivery_location_longitude       0
Order_Date                        0
Time_Orderd                    1731
Time_Order_picked                 0
Weather_conditions              616
Road_traffic_density            601
Vehicle_condition                 0
Type_of_order                     0
Type_of_vehicle                   0
multiple_deliveries             993
Festival                        228
City                           1200
Time_taken (min)                  0
dtype: int64

In [81]:
# data=df.drop(labels=['ID','Delivery_person_ID'], axis=1)
# df = data.loc[data.isnull().sum(1)>=3]
# data = pd.concat([data, df, df]).drop_duplicates(keep=False)
# df = data[data['City'].notna()]

In [24]:
def data_Cleaning(df):
    try:
        data=df.drop(labels=['ID','Delivery_person_ID'], axis=1)
        df = data.loc[data.isnull().sum(1)>=3]
        data = pd.concat([data, df, df]).drop_duplicates(keep=False)
        df = data[data['City'].notna()]
        for i in df.index:
            if df['Time_Orderd'][i] >= '05:00' and df['Time_Orderd'][i] < '10:00':
                df.loc[i,'day_quaters'] = 'morning'
            elif df['Time_Orderd'][i] >= '10:00' and df['Time_Orderd'][i] < '14:00':
                df.loc[i,'day_quaters'] = 'late morning'
            elif df['Time_Orderd'][i] >= '14:00' and df['Time_Orderd'][i] < '19:00':
                df.loc[i,'day_quaters'] = 'afternoon'
            elif df['Time_Orderd'][i] >= '19:00':
                df.loc[i,'day_quaters'] = 'night'
        df=df.drop(labels=['Time_Orderd','Time_Order_picked','Order_Date'], axis=1)
        df['Festival'].fillna("No", inplace = True) # 98% of the column has No as a value 
        df['day_quaters'].fillna("night", inplace = True) # most of the people ordered at night
        df['Delivery_person_Ratings'].fillna(round(np.mean(df['Delivery_person_Ratings']),1), inplace = True)
        df['Delivery_person_Age'].fillna(round(np.mean(df['Delivery_person_Age'])), inplace = True)
        df['multiple_deliveries'].fillna(round(np.mean(df['multiple_deliveries'])), inplace=True)
        df = df.dropna()
        return df
    except Exception as e:
        print("Exception Occured")
df=data_Cleaning(df)
df

C:\Users\akish\AppData\Local\Temp\ipykernel_14424\55342633.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[i,'day_quaters'] = 'night'


,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min),day_quaters
0,36.0,4.2,30.327968,78.046106,30.397968,78.116106,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46,night
1,21.0,4.7,10.003064,76.307589,10.043064,76.347589,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23,afternoon
2,23.0,4.7,18.562450,73.916619,18.652450,74.006619,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21,afternoon
3,34.0,4.3,30.899584,75.809346,30.919584,75.829346,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20,morning
4,24.0,4.7,26.463504,80.372929,26.593504,80.502929,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41,night
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45579,30.0,4.8,26.902328,75.794257,26.912328,75.804257,Windy,High,1,Meal,motorcycle,0.0,No,Metropolitian,32,late morning
45580,21.0,4.6,0.000000,0.000000,0.070000,0.070000,Windy,Jam,0,Buffet,motorcycle,1.0,No,Metropolitian,36,night
45581,30.0,4.9,13.022394,80.242439,13.052394,80.272439,Cloudy,Low,1,Drinks,scooter,0.0,No,Metropolitian,16,night
45582,20.0,4.7,11.001753,76.986241,11.041753,77.026241,Cloudy,High,0,Snack,motorcycle,1.0,No,Metropolitian,26,late morning


In [5]:
df.columns

Index(['Delivery_person_Age', 'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Weather_conditions',
       'Road_traffic_density', 'Vehicle_condition', 'Type_of_order',
       'Type_of_vehicle', 'multiple_deliveries', 'Festival', 'City',
       'Time_taken (min)', 'day_quaters'],
      dtype='object')

In [43]:
# df = df[df['Delivery_person_Age'].notna()]
# df = df[df['Delivery_person_Ratings'].notna()]
# df = df[df['multiple_deliveries'].notna()]
# df = df[df['Festival'].notna()]
df['Festival'].fillna("No", inplace = True) # 98% of the column has No as a value 
df['day_quaters'].fillna("night", inplace = True) # most of the people ordered at night
df['Delivery_person_Ratings'].fillna(round(np.mean(df['Delivery_person_Ratings']),1), inplace = True)
df['Delivery_person_Age'].fillna(round(np.mean(df['Delivery_person_Age'])), inplace = True)
df['multiple_deliveries'].fillna(round(np.mean(df['multiple_deliveries'])), inplace=True)

In [20]:
df.isnull().sum()

Delivery_person_Age            0
Delivery_person_Ratings        0
Restaurant_latitude            0
Restaurant_longitude           0
Delivery_location_latitude     0
Delivery_location_longitude    0
Weather_conditions             0
Road_traffic_density           0
Vehicle_condition              0
Type_of_order                  0
Type_of_vehicle                0
multiple_deliveries            0
Festival                       0
City                           0
Time_taken (min)               0
day_quaters                    0
dtype: int64

In [45]:
df = df.dropna()

In [104]:
data = pd.read_csv("https://raw.githubusercontent.com/AKISHPOTHURI/DataScience/main/Dataset/online_order.csv")

In [79]:
# def data_Cleaning(data):
#     try:
#         df=data.drop(labels=['ID','Delivery_person_ID'], axis=1)
#         df = df.loc[data.isnull().sum(1)>=3]
#         data1 = pd.concat([data, df, df]).drop_duplicates(keep=False)
#         df = data1[data1['City'].notna()]

#         for i in df.index:
#             if df['Time_Orderd'][i] >= '05:00' and df['Time_Orderd'][i] < '10:00':
#                 df.loc[i,'day_quaters'] = 'morning'
#             elif df['Time_Orderd'][i] >= '10:00' and df['Time_Orderd'][i] < '14:00':
#                 df.loc[i,'day_quaters'] = 'late morning'
#             elif df['Time_Orderd'][i] >= '14:00' and df['Time_Orderd'][i] < '19:00':
#                 df.loc[i,'day_quaters'] = 'afternoon'
#             elif df['Time_Orderd'][i] >= '19:00':
#                 df.loc[i,'day_quaters'] = 'night'

#         df['Festival'].fillna("No", inplace = True) # 98% of the column has No as a value 
#         df['Delivery_person_Ratings'].fillna(round(np.mean(df['Delivery_person_Ratings']),1), inplace = True)
#         df['Delivery_person_Age'].fillna(round(np.mean(df['Delivery_person_Age'])), inplace = True)
#         df['multiple_deliveries'].fillna(round(np.mean(df['multiple_deliveries'])), inplace=True)
#         df['day_quaters'].fillna("night", inplace = True) # most of the people ordered at night
#         # df=df.drop(labels=['Time_Orderd','Time_Order_picked','Order_Date'], axis=1)
#         df = df.dropna()
#         return df
#     except Exception as e:
#         print("Exception occured")
# a =data_Cleaning(data)
# a
            

Exception occured


In [72]:
df.isnull().sum()

Delivery_person_Age               0
Delivery_person_Ratings           0
Restaurant_latitude               0
Restaurant_longitude              0
Delivery_location_latitude        0
Delivery_location_longitude       0
Weather_conditions                0
Road_traffic_density              0
Vehicle_condition                 0
Type_of_order                     0
Type_of_vehicle                   0
multiple_deliveries               0
Festival                          0
City                              0
Time_taken (min)                  0
day_quaters                       0
distance                       2716
dtype: int64

In [67]:
data.columns

Index(['ID', 'Delivery_person_ID', 'Delivery_person_Age',
       'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Order_Date', 'Time_Orderd',
       'Time_Order_picked', 'Weather_conditions', 'Road_traffic_density',
       'Vehicle_condition', 'Type_of_order', 'Type_of_vehicle',
       'multiple_deliveries', 'Festival', 'City', 'Time_taken (min)'],
      dtype='object')

In [11]:

# Set the earth's radius (in kilometers)
R = 6371
index = []
# Convert degrees to radians
def deg_to_rad(degrees):
    return degrees * (np.pi/180)

# Function to calculate the distance between two points using the haversine formula
def distcalculate(lat1, lon1, lat2, lon2):
    d_lat = deg_to_rad(lat2-lat1)
    d_lon = deg_to_rad(lon2-lon1)
    a = np.sin(d_lat/2)**2 + np.cos(deg_to_rad(lat1)) * np.cos(deg_to_rad(lat2)) * np.sin(d_lon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c
  
# Calculate the distance between each pair of points
# data1['distance'] = np.nan

for i in range(len(df)):
    if i in df.index:
        df.loc[i, 'distance'] = distcalculate(df.loc[i, 'Restaurant_latitude'], 
                                        df.loc[i, 'Restaurant_longitude'], 
                                        df.loc[i, 'Delivery_location_latitude'], 
                                        df.loc[i, 'Delivery_location_longitude'])
    else:
        index.append(i)



In [25]:
def calc_distance(df):    
    # Set the earth's radius (in kilometers)
    df = df.dropna()
    R = 6371
    index = []
    # Convert degrees to radians
    def deg_to_rad(degrees):
        return degrees * (np.pi/180)

    # Function to calculate the distance between two points using the haversine formula
    def distcalculate(lat1, lon1, lat2, lon2):
        d_lat = deg_to_rad(lat2-lat1)
        d_lon = deg_to_rad(lon2-lon1)
        a = np.sin(d_lat/2)**2 + np.cos(deg_to_rad(lat1)) * np.cos(deg_to_rad(lat2)) * np.sin(d_lon/2)**2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        return R * c
    
    # Calculate the distance between each pair of points
    # data1['distance'] = np.nan

    for i in range(len(df)):
        if i in df.index:
            df.loc[i, 'distance'] = distcalculate(df.loc[i, 'Restaurant_latitude'], 
                                            df.loc[i, 'Restaurant_longitude'], 
                                            df.loc[i, 'Delivery_location_latitude'], 
                                            df.loc[i, 'Delivery_location_longitude'])
        else:
            index.append(i)   
    return df
df = calc_distance(df)
df.head()

,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min),day_quaters,distance
0,36.0,4.2,30.327968,78.046106,30.397968,78.116106,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46,night,10.280582
1,21.0,4.7,10.003064,76.307589,10.043064,76.347589,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23,afternoon,6.242319
2,23.0,4.7,18.562450,73.916619,18.652450,74.006619,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21,afternoon,13.787860
3,34.0,4.3,30.899584,75.809346,30.919584,75.829346,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20,morning,2.930258
4,24.0,4.7,26.463504,80.372929,26.593504,80.502929,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41,night,19.396618


In [26]:
# df = df.dropna()
df.head()

,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min),day_quaters,distance
0,36.0,4.2,30.327968,78.046106,30.397968,78.116106,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46,night,10.280582
1,21.0,4.7,10.003064,76.307589,10.043064,76.347589,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23,afternoon,6.242319
2,23.0,4.7,18.562450,73.916619,18.652450,74.006619,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21,afternoon,13.787860
3,34.0,4.3,30.899584,75.809346,30.919584,75.829346,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20,morning,2.930258
4,24.0,4.7,26.463504,80.372929,26.593504,80.502929,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41,night,19.396618


In [12]:
df.reset_index(drop=True)

,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min),day_quaters,distance
0,36.0,4.2,30.327968,78.046106,30.397968,78.116106,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46,night,10.280582
1,21.0,4.7,10.003064,76.307589,10.043064,76.347589,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23,afternoon,6.242319
2,23.0,4.7,18.562450,73.916619,18.652450,74.006619,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21,afternoon,13.787860
3,34.0,4.3,30.899584,75.809346,30.919584,75.829346,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20,morning,2.930258
4,24.0,4.7,26.463504,80.372929,26.593504,80.502929,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41,night,19.396618
5,29.0,4.5,19.176269,72.836721,19.266269,72.926721,Sandstorms,Jam,2,Buffet,electric_scooter,1.0,No,Metropolitian,20,night,13.763977
6,35.0,4.0,12.311072,76.654878,12.351072,76.694878,Windy,High,1,Meal,scooter,1.0,No,Metropolitian,33,afternoon,6.218001
7,33.0,4.2,18.592718,73.773572,18.702718,73.883572,Sandstorms,Jam,2,Snack,motorcycle,1.0,No,Metropolitian,40,night,16.849940
8,34.0,4.9,17.426228,78.407495,17.496228,78.477495,Cloudy,Jam,0,Snack,motorcycle,1.0,No,Metropolitian,41,night,10.757109
9,21.0,4.7,22.552672,88.352885,22.582672,88.382885,Windy,Jam,0,Meal,motorcycle,1.0,No,Urban,15,night,4.540574


In [27]:
#Independent and dependent features
X = df.drop(labels=['Time_taken (min)'], axis=1)
Y = df[['Time_taken (min)']]

In [28]:
X.head()

,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,day_quaters,distance
0,36.0,4.2,30.327968,78.046106,30.397968,78.116106,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,night,10.280582
1,21.0,4.7,10.003064,76.307589,10.043064,76.347589,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,afternoon,6.242319
2,23.0,4.7,18.562450,73.916619,18.652450,74.006619,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,afternoon,13.787860
3,34.0,4.3,30.899584,75.809346,30.919584,75.829346,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,morning,2.930258
4,24.0,4.7,26.463504,80.372929,26.593504,80.502929,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,night,19.396618


In [29]:
#Define which columns should be ordinal-encoded and which should be scaled
# categorical_cols = X.select_dtypes(include='object').columns
# numerical_cols = X.select_dtypes(exclude='object').columns
categorical_cols = ['Weather_conditions', 'Road_traffic_density', 'Type_of_order',
       'Type_of_vehicle', 'Festival', 'City', 'day_quaters']
#numerical_cols
numerical_cols = ['Delivery_person_Age', 'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Vehicle_condition',
       'multiple_deliveries']

In [30]:
categorical_cols

['Weather_conditions',
 'Road_traffic_density',
 'Type_of_order',
 'Type_of_vehicle',
 'Festival',
 'City',
 'day_quaters']

In [31]:
numerical_cols

['Delivery_person_Age',
 'Delivery_person_Ratings',
 'Restaurant_latitude',
 'Restaurant_longitude',
 'Delivery_location_latitude',
 'Delivery_location_longitude',
 'Vehicle_condition',
 'multiple_deliveries']

In [16]:
df.dtypes

Delivery_person_Age            float64
Delivery_person_Ratings        float64
Restaurant_latitude            float64
Restaurant_longitude           float64
Delivery_location_latitude     float64
Delivery_location_longitude    float64
Weather_conditions              object
Road_traffic_density            object
Vehicle_condition                int64
Type_of_order                   object
Type_of_vehicle                 object
multiple_deliveries            float64
Festival                        object
City                            object
Time_taken (min)                 int64
day_quaters                     object
distance                       float64
dtype: object

In [ ]:
categorical_cols

Index(['Weather_conditions', 'Road_traffic_density', 'Type_of_order',
       'Type_of_vehicle', 'Festival', 'City', 'day_quaters'],
      dtype='object')

In [18]:
city_map = {'Metropolitian','Urban','Semi-Urban'}
Festival_map = {'Yes','No'}
Road_traffic_density_map = {'Jam','High','Medium','Low'}
Type_of_order_map = {'Snack','Meal','Drinks','Buffet'}
Type_of_vehicle_map = {'motorcycle','scooter','electric_scooter'}
Weather_conditions_map = {'Fog','Stormy','Sandstorms','Windy','Cloudy','Sunny'}
day_quaters_map = {'night','afternoon','morning','late morning'}

In [32]:
#Nuerical Pipeline
num_pipeline = Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
    ]
)

#Categorical Pipeline
cat_pipeline = Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('OrdinalEncoder', OrdinalEncoder()),
    ('scaler', StandardScaler())
    ]
)

In [ ]:
numerical_cols

Index(['Delivery_person_Age', 'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Vehicle_condition',
       'multiple_deliveries', 'distance'],
      dtype='object')

In [ ]:
categorical_cols

Index(['Weather_conditions', 'Road_traffic_density', 'Type_of_order',
       'Type_of_vehicle', 'Festival', 'City', 'day_quaters'],
      dtype='object')

In [33]:
preprocessor = ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)
])

In [34]:
preprocessor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['Delivery_person_Age',
                                  'Delivery_person_Ratings',
                                  'Restaurant_latitude', 'Restaurant_longitude',
                                  'Delivery_location_latitude',
                                  'Delivery_location_longitude',
                                  'Vehicle_condition', 'multiple_deliveries']),
                                ('cat_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('OrdinalEncoder',
                                                  OrdinalEncoder()),
                                                 ('scaler', StandardScaler())]),
                                 ['Weather_conditions', 'Road_traffic_density',
                                  'Type_of_order', 'Type_of_vehicle',
                                  'Festival', 'City', 'day_quaters'])])

In [35]:
#Train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.30,random_state=42)

In [ ]:
X_train.shape

(33, 16)

In [ ]:
X_test.shape

(15, 16)

In [36]:
X_train= pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test = pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [37]:
X_train.head()

,num_pipeline__Delivery_person_Age,num_pipeline__Delivery_person_Ratings,num_pipeline__Restaurant_latitude,num_pipeline__Restaurant_longitude,num_pipeline__Delivery_location_latitude,num_pipeline__Delivery_location_longitude,num_pipeline__Vehicle_condition,num_pipeline__multiple_deliveries,cat_pipeline__Weather_conditions,cat_pipeline__Road_traffic_density,cat_pipeline__Type_of_order,cat_pipeline__Type_of_vehicle,cat_pipeline__Festival,cat_pipeline__City,cat_pipeline__day_quaters
0,-0.449203,0.528680,-2.218161,-3.310153,-2.356962,-3.310265,0.003646,0.436253,1.480625,0.292981,0.438968,1.260637,-0.138554,1.836469,0.807480
1,-0.274923,-0.108790,1.251159,0.242727,1.290205,0.241617,0.003646,0.436253,0.309091,-1.838504,-0.454938,1.260637,-0.138554,-0.547189,-0.765383
2,1.119313,-1.702466,-0.540310,0.450999,-0.587715,0.452229,1.228020,0.436253,0.309091,-0.772761,-1.348843,1.260637,-0.138554,-0.547189,0.807480
3,0.945033,0.847415,0.510313,0.100518,0.514873,0.100817,1.228020,0.436253,-0.862443,1.358724,0.438968,-2.120290,-0.138554,-0.547189,-1.551814
4,0.073636,0.209945,0.802790,0.688434,0.821215,0.688254,0.003646,-1.322052,0.894858,0.292981,-0.454938,-0.429827,-0.138554,-0.547189,0.807480


In [28]:
X_test.head()

,num_pipeline__Delivery_person_Age,num_pipeline__Delivery_person_Ratings,num_pipeline__Restaurant_latitude,num_pipeline__Restaurant_longitude,num_pipeline__Delivery_location_latitude,num_pipeline__Delivery_location_longitude,num_pipeline__Vehicle_condition,num_pipeline__multiple_deliveries,num_pipeline__distance,cat_pipeline__Weather_conditions,cat_pipeline__Road_traffic_density,cat_pipeline__Type_of_order,cat_pipeline__Type_of_vehicle,cat_pipeline__Festival,cat_pipeline__City,cat_pipeline__day_quaters
0,-1.712907,0.401459,0.275076,0.079293,0.271065,0.077898,-0.184553,-1.308154,-0.771821,1.752460,-1.933510,1.055560,1.231136,0.0,1.602687,-1.843528
1,0.212125,1.047711,1.006611,0.693057,1.002546,0.691578,-0.184553,0.290701,-0.789255,-0.761939,1.611258,1.055560,-0.331460,0.0,-0.647895,-1.843528
2,0.562131,1.047711,0.268637,0.080300,0.269221,0.080498,-0.184553,0.290701,0.112618,1.123860,1.611258,1.055560,-0.331460,0.0,-0.647895,-1.843528
3,-0.312884,0.724585,-0.887382,0.248731,-0.885181,0.249438,-1.402606,0.290701,0.447372,-0.133339,-0.751921,-0.686114,-0.331460,0.0,-0.647895,0.656599
4,0.387128,-3.476051,-2.572711,-3.840340,-2.565792,-3.837482,-0.184553,-1.308154,1.377385,0.495261,-0.751921,1.055560,1.231136,0.0,1.602687,0.656599


In [ ]:
categorical_cols

Index(['Delivery_person_ID', 'Order_Date', 'Time_Orderd', 'Time_Order_picked',
       'Weather_conditions', 'Road_traffic_density', 'Type_of_order',
       'Type_of_vehicle', 'Festival', 'City'],
      dtype='object')

In [ ]:
numerical_cols

Index(['Delivery_person_Age', 'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Vehicle_condition',
       'multiple_deliveries'],
      dtype='object')

In [29]:
X.isnull().sum()

Delivery_person_Age            0
Delivery_person_Ratings        0
Restaurant_latitude            0
Restaurant_longitude           0
Delivery_location_latitude     0
Delivery_location_longitude    0
Weather_conditions             0
Road_traffic_density           0
Vehicle_condition              0
Type_of_order                  0
Type_of_vehicle                0
multiple_deliveries            0
Festival                       0
City                           0
day_quaters                    0
distance                       0
dtype: int64

In [38]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [39]:
regression = LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [40]:
regression.coef_

array([[ 2.15839864e+00, -2.35223064e+00,  1.21234783e-01,
        -1.67221266e+03, -1.92701287e-01,  1.67232593e+03,
        -1.76185127e+00,  1.95367580e+00, -1.53668888e+00,
        -2.32494062e+00,  1.58674038e-02, -3.03697049e-01,
         1.49011244e+00, -9.44290695e-01, -1.29975682e+00]])

In [41]:
regression.intercept_

array([26.40685848])

In [42]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [43]:
## Train multiple models
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    # print('\n')

LinearRegression
Model Training Performance
RMSE: 6.498243950816648
MAE: 5.212099728019383
R2 score 52.03737993638129
Lasso
Model Training Performance
RMSE: 7.45388191899078
MAE: 6.000597208831721
R2 score 36.893238434841294
Ridge
Model Training Performance
RMSE: 6.9933094405531095
MAE: 5.615231778945197
R2 score 44.45098431726343
Elasticnet
Model Training Performance
RMSE: 7.472335903037435
MAE: 6.045646524495027
R2 score 36.58037789313332


In [38]:
def alpha():
    
    def sum(a,b):
        return a+b
    return sum(a=5,b=5)
alpha()

10